In [37]:
from autogen import ConversableAgent, UserProxyAgent
from dotenv import load_dotenv
import os


In [38]:
load_dotenv()
llm_config = {
    "model": "gpt-4o",
    "api_key": os.environ.get("OPENAI_API_KEY")
}

# Creating Agents

### Preference Collecting Agents

In [39]:
preference_event_type_agent = ConversableAgent(
    name="Event_Type_Preference_Agent",
    system_message="""
    You are an agent that gets the user's name and the type of event that the user wants to plan.
    Do not ask the user for any other information and do not reply to the user.
    Return 'TERMINATE' when you have gathered all the information you need.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER', 
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

In [40]:

preference_event_participant_agent = ConversableAgent(
    name="Event_Participant_Preference_Agent",
    system_message="""
    You are an agent that gets the number of participants of the event that the user wants to plan.
    Adress the user by their name and ask how many participants will be attending the event.
    Do not ask the user for any other information and do not reply to the user.
    Return 'TERMINATE' when you have gathered all the information you need.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER',
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

In [41]:
preference_event_budget_agent = ConversableAgent(
    name="Event_Budget_Preference_Agent",
    system_message="""
    You are an agent that gets the budget per person for the participants of the event.
    Address the user by their name and ask ask if the user prefers to input a total budget, if so, ask for the total budget.
    Do not ask the user for any other information and do not reply to the user.
    Return 'TERMINATE' when you have gathered all the information you need.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER',
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

In [42]:

preference_event_time_agent = ConversableAgent(
    name="Event_Time_Preference_Agent",
    system_message="""
    You are an agent that gets the dates the user wants to plan. The dates can be a range or a specific date.
    Address the user by their name and ask for the dates.
    If the user provides a range, ask for the start and end dates.
    If the user provides a specific date, ask for the date.
    Also, ask for the time of the day for the event.
    Do not ask the user for any other information and do not reply to the user.
    Return 'TERMINATE' when you have gathered all the information you need.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER',
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)


In [43]:
preference_event_location_agent = ConversableAgent(
    name="Event_Location_Preference_Agent",
    system_message="""
    You are an agent that gets the location of the event that the user wants to plan.
    The user will provide a general location, such as a city or a street, or a specific location.
    Do not ask the user for any other information.
    Return 'TERMINATE' when you have gathered all the information you need.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER',
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

In [44]:
preference_event_request_agent = ConversableAgent(
    name="Event_Request_Preference_Agent",
    system_message="""
    You are an agent that gets the user's special requests for the event they want to plan if they have any.
    The user will provide a request, such as "vegan food" or "wheelchair accessibility".
    If the user does not have any special requests, you can return 'TERMINATE'.
    Do not ask the user for any other information.
    Return 'TERMINATE' when you have gathered all the information you need.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER',
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)


In [45]:
preference_proxy_agent = UserProxyAgent(
    name="Event_Preference_Proxy_Agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)


In [46]:

preference_recommendation_agent = ConversableAgent(
    name="Event_Recommendation_Agent",
    system_message="""
    You are an agent that provides recommendations for the event based on the user's preferences.
    You will receive the user's preferences from the Event_Preference_Proxy_Agent.
    Do not ask the user for any other information and do not reply to the user.
    Return 'TERMINATE' when you have provided all the recommendations you can.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER', 
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)



# Creating Tasks

In [47]:
chats = [
    {
        "sender": preference_event_type_agent,
        "recipient": preference_proxy_agent,
        "message": "Hello, welcome to the Event Planner. What's your name and What type of event would you like to plan?",
        "summary_method": "reflection_with_llm",

        "summary_args": {
            "summary_prompt" : "Return the user name and event type the user wants to plan as a JSON object only: "
                             "{'name': '', 'type': ''}",
        },
        "max_turns": 1,
        "clear_history": False, 
    },
    {
        "sender": preference_event_participant_agent,
        "recipient": preference_proxy_agent,
        "message": "What is the number of participants that will be attending the event?",
        "summary_method": "reflection_with_llm",

        "summary_args": {
            "summary_prompt" : "Return the number of participants as a JSON object only: "
                             "{'number_participants': 0}",
        },
        "max_turns": 1,
        "clear_history": False, 
    },
    {
        "sender": preference_event_budget_agent,
        "recipient": preference_proxy_agent,
        "message": "What is your budget per person for the event? If you prefer to input a total budget, please specify that.",
        "summary_method": "reflection_with_llm",

        "summary_args": {
            "summary_prompt" : "Return the budget per person and total budget as a JSON object only: "
                             "{'budget_per_person': 0, 'total_budget': 0}",
        },
        "max_turns": 1,
        "clear_history": False, 
    },
    {
        "sender": preference_event_time_agent,
        "recipient": preference_proxy_agent,
        "message": "What dates do you want to plan the event for? Please provide a range or a specific date, and also specify the time of day.",
        "summary_method": "reflection_with_llm",

        "summary_args": {
            "summary_prompt" : "Return the event dates and time as a JSON object only: "
                             "{'start_date': '', 'end_date': '', 'time_of_day': ''}",
        },
        "max_turns": 1,
        "clear_history": False, 
    },
    {
        "sender": preference_event_location_agent,
        "recipient": preference_proxy_agent,
        "message": "What is the location of the event? You can provide a general location or a specific address.",
        "summary_method": "reflection_with_llm",

        "summary_args": {
            "summary_prompt" : "Return the event location as a JSON object only: "
                             "{'location': ''}",
        },
        "max_turns": 1,
        "clear_history": False, 
    },
    {
        "sender": preference_event_request_agent,
        "recipient": preference_proxy_agent,
        "message": "Do you have any special requests for the event? If not, you can say 'no'.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the special requests as a JSON object only: "
                             "{'special_requests': ''}",
        },
        "clear_history": False,
    },
    {
        "sender": preference_proxy_agent,
        "recipient": preference_recommendation_agent,
        "message": "Here are the user's preferences for the event. Please provide recommendations based on these preferences.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": False,
    }
]

In [48]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Event_Type_Preference_Agent (to Event_Preference_Proxy_Agent):

Hello, welcome to the Event Planner. What's your name and What type of event would you like to plan?

--------------------------------------------------------------------------------
Event_Preference_Proxy_Agent (to Event_Type_Preference_Agent):

Ersoy and BBQ

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (af706f97-8a08-4de4-ba93-e659af29aa06): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
Event_Participant_Preference_Agent (to Event_Preference_Proxy_Agent):

What is the number of participants that will be attend

KeyboardInterrupt: Interrupted by user